In [13]:
import pygame
import neat #neuroevolution
import os
import random
import cv2

In [14]:
pygame.font.init() #Initialize

In [15]:
img_bgr = cv2.imread('images/brg.jpg')
# img_bgr = cv2.resize(img_bgr, (900, 168*3))
# cv2.imwrite('images/brg.jpg', img_bgr)
WIN_HEIGHT = img_bgr.shape[0]
WIN_WIDTH = img_bgr.shape[1]
DRAW_LINES = True
GEN = 0

In [16]:
pygame.display.set_caption('Flappy Bird')

In [17]:
BIRD_IMGS = [pygame.transform.scale2x(pygame.image.load('images/bird1.png')),
            pygame.transform.scale2x(pygame.image.load('images/bird2.png')),
            pygame.transform.scale2x(pygame.image.load('images/bird3.png'))]
BASE_IMG = pygame.transform.scale2x(pygame.image.load('images/base.png'))
BRG_IMG = pygame.image.load('images/brg.jpg')
PIPE_IMG = pygame.transform.scale2x(pygame.image.load('images/pipe.png'))          

In [18]:
STAT_FONT = pygame.font.SysFont('comicsans', 50) 

In [19]:
class Bird:
    ROTATION_VEL = 20 # tốc độ xoay của bird khi nhảy
    MAX_ROTATION = 50 # tốc độ xoay tối đa
    ANIMATION_TIME = 5 # thời gian giữa các frame khi bird đang bay
    IMGS = BIRD_IMGS 

    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.height = self.y
        self.frame_count = 0 # đếm số frame đã trôi qua từ khi con chim nhảy
        self.tilt = 0 # Góc nghiêng của bird, dùng điều chỉnh tạo hiệu ứng xoay khi bird nhảy
        self.vel = 0 #tốc độ di chuyển của bird
        self.img_number = 0 # STT ảnh trong IMGS
        self.img = self.IMGS[0] 

    def jump(self):
        self.vel = -10.5 #
        self.frame_count = 0
        self.height = self.y
    
    def move(self): #cập nhật vị trí và góc nghiêng dựa trên thời gian và tốc độ(velocity)
        self.frame_count += 1 # theo dõi thời gian trôi qua
        d = self.vel * self.frame_count + 1.5*self.frame_count ** 2 #Tính vị trí mới dựa trên pt quỹ đạo ch động đồng thời với gia tốc
        if d >= 16: #ngăn rơi xuống quá nhanh
            d = 16
        
        self.y = self.y + d # cập nhật tọa độ y

        if d<0 or self.y < self.height + 50: #
            if self.tilt < self.MAX_ROTATION: #
                self.tilt = self.MAX_ROTATION # Tạo hiệu ứng nghiêng về phía trước khi đang đi lên
        else:
            if self.tilt > -90:
                self.tilt -= self.ROTATION_VEL # Tạo hiệu ứng nghiêng xg khi đang rơi xống
    
    def draw(self, win):
        self.img_number += 1 #Tạo hiệu ứng vỗ cánh
        
        if self.img_number < self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_number < self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.img_number < self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.img_number < self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.img_number < self.ANIMATION_TIME*4 + 1:
            self.img = self.IMGS[0]
            self.img_number = 0
        
        if self.tilt < -80:
            self.img = self.IMGS[1]
            self.img_number = self.ANIMATION_TIME*2
        
        rotated_img = pygame.transform.rotate(self.img, self.tilt) #qoay ảnh tạo hiệu ứng xoay
        new_rect = rotated_img.get_rect(center=self.img.get_rect(topleft=(self.x, self.y)).center) #
        win.blit(rotated_img, new_rect)
    
    def get_mask(self):
        return pygame.mask.from_surface(self.img)
        

In [20]:
class Pipe:
    GAP = 200
    VEL = 5

    def __init__(self, x):
        self.x = x
        self.height = 0
        self.top = 0
        self.bottom = 0
        self.TOP_PIPE = pygame.transform.flip(PIPE_IMG, False, True)
        self.BOTTOM_PIPE = PIPE_IMG
        self.passed = False
        self.set_height()
    
    def set_height(self):
        self.height = random.randrange(50, img_bgr.shape[0]-250)
        self.bottom = self.height + self.GAP # tính tọa độ y của pipe dưới
        self.top = self.height - self.TOP_PIPE.get_height() # tính tọa độ của pipe trên dựa trên chiều cao của pipe trên
    
    def move(self):
        self.x -= self.VEL #di chuyển pipe sang trái dựa trên tốc độ VEL

    #hiển thị cặp pipe
    def draw(self, win):
        win.blit(self.TOP_PIPE, (self.x, self.top))
        win.blit(self.BOTTOM_PIPE, (self.x, self.bottom))

    # Kiểm tra va chạm giữa bird và pipe
    def collide(self, bird):
        bird_mask = bird.get_mask() # lấy mask của bird từ class bird
        top_pipe_mask = pygame.mask.from_surface(self.TOP_PIPE) # lấy mask của ống trên
        bottom_pipe_mask = pygame.mask.from_surface(self.BOTTOM_PIPE)

        top_offset = (self.x - bird.x, self.top - round(bird.y)) # Tính offset giữa ống trên và bird
        bottom_offset = (self.x - bird.x, self.bottom - round(bird.y))

        top_overlap = bird_mask.overlap(top_pipe_mask, top_offset) # Kiểm tra sự va chạm giữa bird và pipe
        bottom_overlap = bird_mask.overlap(bottom_pipe_mask, bottom_offset)

        if top_overlap or bottom_overlap:
            return True
        return False

In [21]:
class Base:
    VEL = 5
    IMG = BASE_IMG
    WIDTH = BASE_IMG.get_width()

    def __init__(self, y):
        self.y = y
        self.x1 = 0
        self.x2 = self.WIDTH
    
    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL

        # tạo hiệu ứng lặp
        if self.x1 < -self.WIDTH:
            self.x1 = self.x2 + self.WIDTH
        if self.x2 < -self.WIDTH:
            self.x2 = self.x1 + self.WIDTH
    
    # Hiện base lên màn hình
    def draw(self, win):
        win.blit(self.IMG, (self.x1, self.y))
        win.blit(self.IMG, (self.x2, self.y))

In [22]:
def draw_window(win, birds, pipes, base, score, GEN, pipe_ind):
    global DRAW_LINES
    win.blit(BRG_IMG, (0, 0))
    for pipe in pipes:
        pipe.draw(win)
    base.draw(win)

    for bird in birds:
        # if DRAW_LINES:
        #     try:
        #         pygame.draw.line(win, (255, 0, 0),
        #                         (bird.x + bird.img.get_width()/2, bird.y + bird.img.get_height()/2),
        #                         (pipes[pipe_ind].x + pipes[pipe_ind].TOP_PIPE.get_width()/2, pipes[pipe_ind].height), 5)

        #         pygame.draw.line(win, (255, 0, 0),
        #                         (bird.x + bird.img.get_width()/2, bird.y + bird.img.get_height()/2),
        #                         (pipes[pipe_ind].x + pipes[pipe_ind].BOTTOM_PIPE.get_width()/2, pipes[pipe_ind].bottom), 5)
        #     except:
        #         pass
        bird.draw(win)
    text = STAT_FONT.render('Score: ' + str(score), 1, (255, 255, 255))
    win.blit(text, (WIN_WIDTH - 10 - text.get_width(), 10))

    text = STAT_FONT.render('Gen: ' + str(GEN), 1, (255, 255, 255))
    win.blit(text, (10, 5))

    text = STAT_FONT.render('Alive: ' + str(len(birds)), 1, (255, 255, 255))
    win.blit(text, (10, 50))

    pygame.display.update()


In [23]:
def main(genomes, config):
    global GEN
    GEN += 1
    birds = []
    ge = []
    neural_networks = []

    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g, config)
        neural_networks.append(net)
        birds.append(Bird(230, 350))
        g.fitness = 0
        ge.append(g)
    
    pipes = [Pipe(600)]
    base_object = Base(630)
    win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
    clock = pygame.time.Clock()
    run = True
    score = 0

    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_q:
                    run = False
                    pygame.quit()
        
        pipe_ind = 0

        if len(birds) > 0:
            if len(pipes) > 1 and birds[0].x > pipes[0].x + pipes[0].TOP_PIPE.get_width():
                pipe_ind = 1
        else:
            break

        for x, bird in enumerate(birds):
            bird.move()
            ge[x].fitness += 0.1
            output = neural_networks[x].activate(
                (bird.y, abs(bird.y - pipes[pipe_ind].height),
                abs(bird.y - pipes[pipe_ind].bottom))
            )
            if output[0] > 0.5:
                bird.jump()
        
        for pipe in pipes:
            for x, bird in enumerate(birds):
                if pipe.collide(bird) or (bird.y + bird.img.get_height() >= 630 or bird.y < 0):
                    ge[x].fitness -= 1
                    birds.pop(x)
                    ge.pop(x)
                    neural_networks.pop(x)

                if not pipe.passed and bird.x > pipe.x:
                    pipe.passed = True
                    for g in ge:
                        g.fitness += 5
                    score += 1
                    pipes.append(Pipe(500))
            if pipe.x + pipe.TOP_PIPE.get_width() < 0:
                pipes.remove(pipe)
            pipe.move()

        base_object.move()
        draw_window(win, birds, pipes, base_object, score, GEN, pipe_ind)

In [24]:
def run(config_path):
    config = neat.config.Config(
        neat.DefaultGenome, neat.DefaultReproduction,
        neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path
    )

    population = neat.Population(config)
    population.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    population.add_reporter(stats)

    winner = population.run(main, 50)
    print('\nBest genome:\n{!s}'.format(winner))

if __name__ == '__main__':
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, 'config-feedforward.txt')
    run(config_path)


 ****** Running generation 0 ****** 

Population's average fitness: 2.92200 stdev: 3.14915
Best fitness: 17.10000 - size: (1, 3) - species 1 - id 74
Average adjusted fitness: 0.146
Mean genetic distance 1.091, standard deviation 0.351
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100     17.1    0.146     0
Total extinctions: 0
Generation time: 4.924 sec

 ****** Running generation 1 ****** 

Population's average fitness: 4.11100 stdev: 8.91337
Best fitness: 89.30000 - size: (1, 3) - species 1 - id 104
Average adjusted fitness: 0.041
Mean genetic distance 1.087, standard deviation 0.446
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100     89.3    0.041     0
Total extinctions: 0
Generation time: 17.546 sec (11.235 average)

 ****** Running generation 2 ****** 



error: display Surface quit

: 